### 中醫醫案症狀詞抽取
- model: CRF
- 主症與兼症合併
- 詞位標注: OS
- 671個詞沒有完全對應(皆標為O)
- postag使用nltk
- 以句點為分隔拆句子


In [1]:
import json
import nltk

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [3]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i >= ne_idx and i <= (ne_idx + ne_len - 1):
            return "S"
    
    return "O"

In [4]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2].replace(" ","")
    sentences = case_content.split("。")
    
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    #print(summary_str)
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    t += len(main_list)
        
        
    for sent in sentences:
        ne_index_list = []
        for ne in main_list:
            ne_idx = sent.find(ne)
            if ne_idx != -1 :
                ne_index_list.append((ne_idx,len(ne)))

        texts = []
        for i,w in enumerate(sent):
            label = getLabelWithWordIndex(i,ne_index_list)
            texts.append((w,label))

        docs.append(texts)

print(len(docs))

48901


In [5]:
data = []

for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    tagged = nltk.pos_tag(tokens)
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

In [6]:
print(len(data))

48901


In [7]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [8]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
print(len(X_train))
print(len(X_test))

39120
9781


In [10]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 25291
Seconds required: 3.053

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 292754.553036
Feature norm: 1.000000
Error norm: 245497.636818
Active features: 25027
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 0.666

***** Iteration #2 *****
Loss: 274201.957174
Feature norm: 0.923740
Error norm: 228715.692000
Active features: 23513
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.343

***** Iteration #3 *****
Loss: 258834.333001
Feature norm: 0.460758
Error norm: 547128.317495
Active features: 17605
Line search trials: 2
Line search step: 0.500000
Seconds requir

***** Iteration #39 *****
Loss: 51862.852886
Feature norm: 24.481300
Error norm: 12649.139682
Active features: 19723
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.348

***** Iteration #40 *****
Loss: 50621.490409
Feature norm: 27.215471
Error norm: 29279.229073
Active features: 19608
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.339

***** Iteration #41 *****
Loss: 49531.358966
Feature norm: 28.686735
Error norm: 13433.819432
Active features: 19550
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.347

***** Iteration #42 *****
Loss: 48461.197587
Feature norm: 31.357597
Error norm: 11857.757963
Active features: 19522
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.336

***** Iteration #43 *****
Loss: 47324.185433
Feature norm: 34.648254
Error norm: 9383.008498
Active features: 19364
Line search trials: 1
Line search step: 1.00

***** Iteration #81 *****
Loss: 38845.232995
Feature norm: 76.820533
Error norm: 8348.702214
Active features: 17930
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.336

***** Iteration #82 *****
Loss: 38802.074846
Feature norm: 77.056133
Error norm: 3863.716660
Active features: 17935
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.349

***** Iteration #83 *****
Loss: 38753.963491
Feature norm: 77.442655
Error norm: 5936.035082
Active features: 17912
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.343

***** Iteration #84 *****
Loss: 38712.523736
Feature norm: 77.688860
Error norm: 6176.354597
Active features: 17873
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.338

***** Iteration #85 *****
Loss: 38678.135410
Feature norm: 77.951548
Error norm: 6695.298830
Active features: 17886
Line search trials: 1
Line search step: 1.000000

***** Iteration #121 *****
Loss: 37926.906558
Feature norm: 82.700618
Error norm: 3151.459350
Active features: 17581
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.336

***** Iteration #122 *****
Loss: 37919.911610
Feature norm: 82.801260
Error norm: 5410.389409
Active features: 17592
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.337

***** Iteration #123 *****
Loss: 37907.839911
Feature norm: 82.930513
Error norm: 2401.037582
Active features: 17615
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.340

***** Iteration #124 *****
Loss: 37899.833883
Feature norm: 82.985892
Error norm: 2149.592343
Active features: 17610
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.346

***** Iteration #125 *****
Loss: 37889.700601
Feature norm: 83.111147
Error norm: 2613.656450
Active features: 17586
Line search trials: 1
Line search step: 1.0

***** Iteration #161 *****
Loss: 37646.330223
Feature norm: 84.828760
Error norm: 343.972972
Active features: 17388
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.338

***** Iteration #162 *****
Loss: 37641.695793
Feature norm: 84.860843
Error norm: 270.711413
Active features: 17351
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.344

***** Iteration #163 *****
Loss: 37640.149048
Feature norm: 84.866962
Error norm: 1490.661379
Active features: 17338
Line search trials: 4
Line search step: 0.125000
Seconds required for this iteration: 1.348

***** Iteration #164 *****
Loss: 37638.095866
Feature norm: 84.873677
Error norm: 2983.456606
Active features: 17316
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.338

***** Iteration #165 *****
Loss: 37634.186489
Feature norm: 84.893111
Error norm: 1081.709673
Active features: 17321
Line search trials: 1
Line search step: 1.000

In [11]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

舌 (O)
质 (O)
淡 (O)
红 (O)
， (O)
苔 (O)
薄 (O)
白 (O)
， (O)
指 (O)
纹 (O)
淡 (O)
紫 (O)


In [12]:
import numpy as np
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"O": 0,"S": 1}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["O", "S"]))

             precision    recall  f1-score   support

          O       0.97      0.99      0.98    304217
          S       0.58      0.41      0.48     13646

avg / total       0.96      0.96      0.96    317863

